In [2]:
import pandas as pd
import folium
import geopandas as gpd
from shapely.geometry import Point

Raw Dataframe (2020년 8월 위성 데이터 중 이미 행정동 GeoJSON으로 필터링된 일사량 데이터 사용)

In [3]:
df = pd.read_csv("2020-08_insol_filtered.csv", encoding="UTF-8")
#C:\Users\HOME\Documents\workspace\solar-project\Data\monthly_insolation_2023.11-2024.12.csv

#df.columns = ["latitude", "longitude","insolation per day (kmh/m^2)"]
df

,Unnamed: 0,latitude,longitude,insolation per day (kmh/m^2),insolation per month (kmh/m^2),coord,city
0,101,38.4484,128.2927,2.9425,91.2175,"(38.4484, 128.2927)",강원특별자치도 고성군
1,102,38.4483,128.2986,2.9376,91.0656,"(38.4483, 128.2986)",강원특별자치도 고성군
2,103,38.4481,128.3044,2.9286,90.7866,"(38.4481, 128.3044)",강원특별자치도 고성군
3,104,38.4480,128.3103,2.9194,90.5014,"(38.448, 128.3103)",강원특별자치도 고성군
4,105,38.4479,128.3162,2.9195,90.5045,"(38.4479, 128.3162)",강원특별자치도 고성군
...,...,...,...,...,...,...,...
386022,1228842,33.1708,126.2741,4.8941,151.7171,"(33.1708, 126.2741)",제주특별자치도 서귀포시
386023,1229935,33.1663,126.2686,4.9142,152.3402,"(33.1663, 126.2686)",제주특별자치도 서귀포시
386024,1229936,33.1663,126.2741,4.9096,152.1976,"(33.1663, 126.2741)",제주특별자치도 서귀포시
386025,1240878,33.1206,126.2685,5.0999,158.0969,"(33.1206, 126.2685)",제주특별자치도 서귀포시


In [4]:
#지도 생성

m = folium.Map(location=[36.5, 127.5], zoom_start=7)

#각 도시를 색상으로 표시
#행정동 경계 데이터 파일 링크 (https://github.com/vuski/admdongkor/blob/master/ver20241001/HangJeongDong_ver20241001.geojson)

geojson_path = "..\\Data\\HangJeongDong_ver20241001.geojson"

gdf = gpd.read_file(geojson_path)

도시(광역시, 도) 명을 기준으로 8월 합계 일사량 평균값 산출하기

In [5]:
#df에서 "insolation per month (kmh/m^2)" 행과 "city" 행만 남기기

insol_df = df[["city", "insolation per month (kmh/m^2)"]]
insol_df = insol_df.reset_index(drop=True)
insol_df


,city,insolation per month (kmh/m^2)
0,강원특별자치도 고성군,91.2175
1,강원특별자치도 고성군,91.0656
2,강원특별자치도 고성군,90.7866
3,강원특별자치도 고성군,90.5014
4,강원특별자치도 고성군,90.5045
...,...,...
386022,제주특별자치도 서귀포시,151.7171
386023,제주특별자치도 서귀포시,152.3402
386024,제주특별자치도 서귀포시,152.1976
386025,제주특별자치도 서귀포시,158.0969


좌표별 일사량 데이터를 광역시도명으로 분류 후 평균하기

In [6]:
insol_df["SiDo"] = insol_df["city"].str.split().str[0]
insol_df


,city,insolation per month (kmh/m^2),SiDo
0,강원특별자치도 고성군,91.2175,강원특별자치도
1,강원특별자치도 고성군,91.0656,강원특별자치도
2,강원특별자치도 고성군,90.7866,강원특별자치도
3,강원특별자치도 고성군,90.5014,강원특별자치도
4,강원특별자치도 고성군,90.5045,강원특별자치도
...,...,...,...
386022,제주특별자치도 서귀포시,151.7171,제주특별자치도
386023,제주특별자치도 서귀포시,152.3402,제주특별자치도
386024,제주특별자치도 서귀포시,152.1976,제주특별자치도
386025,제주특별자치도 서귀포시,158.0969,제주특별자치도


In [7]:
#같은 광역시/도에 있는 8월 합계 일사량끼리 평균하기


sido_insol_df = insol_df.groupby("SiDo")["insolation per month (kmh/m^2)"].mean().reset_index()
sido_insol_df #도시별 모든 좌표에서 평균한 8월 합계 일사량 데이터 프레임 산출

,SiDo,insolation per month (kmh/m^2)
0,강원특별자치도,93.482396
1,경기도,95.977855
2,경상남도,127.091002
3,경상북도,121.851045
4,광주광역시,128.121633
5,대구광역시,129.456696
6,대전광역시,119.852851
7,부산광역시,135.749932
8,서울특별시,94.958402
9,세종특별자치시,115.940836


2020년 8월의 월 합계 입사량 데이터를 이용하여 광역시도별 컬러맵으로 시각화하기

In [8]:
from branca.colormap import linear
from matplotlib.colors import Normalize, rgb2hex
from matplotlib import colormaps 

#일사량을 컬러맵으로 매핑

min_sido_insol = sido_insol_df['insolation per month (kmh/m^2)'].min()
max_sido_insol = sido_insol_df['insolation per month (kmh/m^2)'].max()
colormap = colormaps["coolwarm"]


norm = Normalize(vmin = min_sido_insol, vmax = max_sido_insol)
print(min_sido_insol, max_sido_insol) #최소, 최대 일사량 값 확인

sido_insol_df["Color(hex)"] = sido_insol_df["insolation per month (kmh/m^2)"].apply(lambda insol: rgb2hex(colormap(norm(insol)))) 
# 일사량 값을 정규화 하여 컬러맵으로 변환

sido_insol_df

93.48239612646104 135.7499319587629


,SiDo,insolation per month (kmh/m^2),Color(hex)
0,강원특별자치도,93.482396,#3b4cc0
1,경기도,95.977855,#4c66d6
2,경상남도,127.091002,#ee8669
3,경상북도,121.851045,#f7b79b
4,광주광역시,128.121633,#ea7b60
5,대구광역시,129.456696,#e36c55
6,대전광역시,119.852851,#f4c5ad
7,부산광역시,135.749932,#b40426
8,서울특별시,94.958402,#445acc
9,세종특별자치시,115.940836,#e5d8d1


In [9]:
gdf

,adm_nm,adm_cd2,sgg,sido,sidonm,sggnm,adm_cd,geometry
0,서울특별시 종로구 사직동,1111053000,11110,11,서울특별시,종로구,11010530,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,서울특별시 종로구 삼청동,1111054000,11110,11,서울특별시,종로구,11010540,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,서울특별시 종로구 부암동,1111055000,11110,11,서울특별시,종로구,11010550,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,서울특별시 종로구 평창동,1111056000,11110,11,서울특별시,종로구,11010560,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,서울특별시 종로구 무악동,1111057000,11110,11,서울특별시,종로구,11010570,"MULTIPOLYGON (((126.96067 37.5808, 126.96281 3..."
...,...,...,...,...,...,...,...,...
3549,전북특별자치도 부안군 줄포면,5280041000,52800,52,전북특별자치도,부안군,35580410,"MULTIPOLYGON (((126.7217 35.57303, 126.72142 3..."
3550,전북특별자치도 부안군 위도면,5280042000,52800,52,전북특별자치도,부안군,35580420,"MULTIPOLYGON (((126.24581 35.57657, 126.24583 ..."
3551,전북특별자치도 전주시완산구 효자5동,5211173000,52111,52,전북특별자치도,전주시완산구,35011790,"MULTIPOLYGON (((127.08727 35.84003, 127.09023 ..."
3552,전북특별자치도 전주시덕진구 혁신동,5211367000,52113,52,전북특별자치도,전주시덕진구,35012700,"MULTIPOLYGON (((127.05915 35.83844, 127.06722 ..."


In [10]:
#GeoJSON 데이터 읍, 면, 동, 시 삭제하고 광역시/도 단위로 묶기

gdf_sido_grouped = gdf.dissolve(by = "sidonm", as_index=False)
gdf_sido_grouped


,sidonm,geometry,adm_nm,adm_cd2,sgg,sido,sggnm,adm_cd
0,강원특별자치도,"POLYGON ((127.81837 37.14772, 127.81818 37.147...",강원특별자치도 춘천시 신북읍,5111025000,51110,51,춘천시,32010110
1,경기도,"MULTIPOLYGON (((126.55825 37.06241, 126.55854 ...",경기도 수원시장안구 파장동,4111156000,41111,41,수원시장안구,31011540
2,경상남도,"MULTIPOLYGON (((127.93372 34.72741, 127.93346 ...",경상남도 진주시 문산읍,4817025000,48170,48,진주시,38030110
3,경상북도,"MULTIPOLYGON (((128.16174 35.77795, 128.16121 ...",경상북도 포항시남구 구룡포읍,4711125000,47111,47,포항시남구,37011110
4,광주광역시,"POLYGON ((126.89541 35.0808, 126.89471 35.0801...",광주광역시 동구 충장동,2911052500,29110,29,동구,24010510
5,대구광역시,"POLYGON ((128.52885 35.71189, 128.5295 35.7112...",대구광역시 중구 삼덕동,2711054500,27110,27,중구,22010540
6,대전광역시,"POLYGON ((127.35975 36.26291, 127.35966 36.262...",대전광역시 동구 효동,3011053000,30110,30,동구,25010530
7,부산광역시,"MULTIPOLYGON (((128.97141 35.03449, 128.97182 ...",부산광역시 중구 중앙동,2611051000,26110,26,중구,21010510
8,서울특별시,"POLYGON ((126.88971 37.45232, 126.88962 37.452...",서울특별시 종로구 사직동,1111053000,11110,11,종로구,11010530
9,세종특별자치시,"POLYGON ((127.21068 36.43549, 127.20987 36.435...",세종특별자치시 세종시 조치원읍,3611025000,36110,36,세종시,29010110


In [12]:
#Folium에 사용할 데이터 준비: gdf_sido_grouped데이터와 sido_insol_df 병합

sido_merged = gdf_sido_grouped.merge(sido_insol_df, left_on="sidonm", right_on="SiDo", how = "left")
print(sido_merged.info())
sido_merged

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 11 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   sidonm                          17 non-null     object  
 1   geometry                        17 non-null     geometry
 2   adm_nm                          17 non-null     object  
 3   adm_cd2                         17 non-null     object  
 4   sgg                             17 non-null     object  
 5   sido                            17 non-null     object  
 6   sggnm                           17 non-null     object  
 7   adm_cd                          17 non-null     object  
 8   SiDo                            17 non-null     object  
 9   insolation per month (kmh/m^2)  17 non-null     float64 
 10  Color(hex)                      17 non-null     object  
dtypes: float64(1), geometry(1), object(9)
memory usage: 1.6+ KB
None


,sidonm,geometry,adm_nm,adm_cd2,sgg,sido,sggnm,adm_cd,SiDo,insolation per month (kmh/m^2),Color(hex)
0,강원특별자치도,"POLYGON ((127.81837 37.14772, 127.81818 37.147...",강원특별자치도 춘천시 신북읍,5111025000,51110,51,춘천시,32010110,강원특별자치도,93.482396,#3b4cc0
1,경기도,"MULTIPOLYGON (((126.55825 37.06241, 126.55854 ...",경기도 수원시장안구 파장동,4111156000,41111,41,수원시장안구,31011540,경기도,95.977855,#4c66d6
2,경상남도,"MULTIPOLYGON (((127.93372 34.72741, 127.93346 ...",경상남도 진주시 문산읍,4817025000,48170,48,진주시,38030110,경상남도,127.091002,#ee8669
3,경상북도,"MULTIPOLYGON (((128.16174 35.77795, 128.16121 ...",경상북도 포항시남구 구룡포읍,4711125000,47111,47,포항시남구,37011110,경상북도,121.851045,#f7b79b
4,광주광역시,"POLYGON ((126.89541 35.0808, 126.89471 35.0801...",광주광역시 동구 충장동,2911052500,29110,29,동구,24010510,광주광역시,128.121633,#ea7b60
5,대구광역시,"POLYGON ((128.52885 35.71189, 128.5295 35.7112...",대구광역시 중구 삼덕동,2711054500,27110,27,중구,22010540,대구광역시,129.456696,#e36c55
6,대전광역시,"POLYGON ((127.35975 36.26291, 127.35966 36.262...",대전광역시 동구 효동,3011053000,30110,30,동구,25010530,대전광역시,119.852851,#f4c5ad
7,부산광역시,"MULTIPOLYGON (((128.97141 35.03449, 128.97182 ...",부산광역시 중구 중앙동,2611051000,26110,26,중구,21010510,부산광역시,135.749932,#b40426
8,서울특별시,"POLYGON ((126.88971 37.45232, 126.88962 37.452...",서울특별시 종로구 사직동,1111053000,11110,11,종로구,11010530,서울특별시,94.958402,#445acc
9,세종특별자치시,"POLYGON ((127.21068 36.43549, 127.20987 36.435...",세종특별자치시 세종시 조치원읍,3611025000,36110,36,세종시,29010110,세종특별자치시,115.940836,#e5d8d1


In [13]:
#GeoJSON 스타일 함수 정의

def style_function(feature):
    color = feature['properties'].get('Color(hex)', None)
    if color: #색상이 있는 경우 
        return {
            "fillColor" : color,
            "color" : "black", #경계선 색
            "weight": 1, #경계선 두께
            "fillOpacity": 0.7
        }
    else: #색상 데이터가 없는 경우 투명 처리
        return {
            "fillColor" : "transparent",
            "color" : "black", #경계선 색
            "weight": 1, #경계선 두께
            "fillOpacity": 0
        }

#GeoJSON 레이어 추가
folium.GeoJson(
    sido_merged,
    style_function = style_function,
    tooltip = folium.GeoJsonTooltip(fields=["SiDo", "insolation per month (kmh/m^2)"])
).add_to(m)

In [14]:
m.save("..\\Visualization\\sido_insolation_map(2020-08).html")
print("지도가 'sido_insolation_map(2020-08).html'로 저장되었습니다. 브라우저에서 확인하세요.")

지도가 'sido_insolation_map(2020-08).html'로 저장되었습니다. 브라우저에서 확인하세요.
